# 14-day cumulative number of COVID-19 cases per 100 000

At the end of the page, we provide a detailed description of how the numbers are calculated.

### Compute data

In [ ]:
import datetime as dt
import pandas as pd
pd.set_option("max_rows", None)
from oscovida import fetch_cases, get_population, get_country_data

# get a list of all country names
countries = fetch_cases().index.drop_duplicates()

data = []
yesterday = dt.date.today() - dt.timedelta(days=1) 

for region in countries:
    c, _, _ = get_country_data(region)   # get cumulative infections c
    if c.index[-1].date() < yesterday:
        print(f"{region}: last data is from {c.index[-1].date()}")
        origin = c.index[0].date()
        # Fill data series forward up to yesterday
        new_idx = pd.date_range(origin, periods=(yesterday - origin).days, freq='D')
        c.reindex(new_idx, method='pad')
    c = c[-15:]
    try:
        population = get_population(region)
        new_cases = int(c[-1] - c[-15]) 
        incidence = new_cases / population * 100000. 
        data += [(region, population, new_cases, round(incidence, 1))]
    except ValueError:
        print(f"Skip {region}")    # skip regions for which we have no population numbers

data.sort(key=lambda x: x[3], reverse=True)

# turn into pandas DataFrame for easier display
table = pd.DataFrame(data, columns=["country", "population", "new cases", "14-day-incidence"]).set_index("country")

# Show last update date
import time
print(f"Last updated {time.asctime()}")

## Table for all countries

In [ ]:
table.sort_index()

## Table sorted by 14-day-incidence

In [ ]:
table

## Tutorial: Detailed calculation for one country

In [ ]:
cases, deaths, label = get_country_data("Germany")

Cumulative cases yesterday (numbers for today are not known yet):

In [ ]:
c_y = cases[-1]
c_y

Cumulative cases 15 days ago (i.e. 14 days before yesterday)

In [ ]:
c_15 = cases[-15]
c_15

New cases from the last 14 days

In [ ]:
c_new = c_y - c_15 
c_new

Get the population data

In [ ]:
population = get_population("Germany")
population

Compute the 14-day incidence per 100000, i.e. the cumulative number of new infections in the last 14 days, normalised by the country's population in units of 100000:

In [ ]:
incidence = c_new / (population/100000)
incidence

This should compare to the numbers as they are provided for example at https://www.ecdc.europa.eu/en/cases-2019-ncov-eueea (assuming the infection numbers from ECDE and JHU are identical, and we are looking at the same dates).

---------------

In [ ]:
# For debugging: When was the data downloaded
import oscovida
oscovida.fetch_cases_last_execution()

In [ ]:
oscovida.display_binder_link('14-day-incidence.ipynb')